This notebook used resting-state time-series to calculate the subject-level connectivity matrix of the Cognitive Control network as identified in [@dosenbach2007]. The connectivity matrix, of shape $N_{\text{subjects}} \times N_{\text{regions}} \times N_{\text{regions}}$, will be stored as an annotated NetCDF4 file in the following file: `data/julia2018_resting/connectivities.nc`.

In [2]:
import numpy as np
import xarray as xr
from nilearn.connectome import ConnectivityMeasure

In [40]:
DATASET = xr.open_dataset('data/julia2018_resting/timeseries.nc')
TIMESERIES = DATASET['timeseries']

In [43]:
def fit_connectivity(timeseries: np.ndarray, kind='correlation'):
  """Extract a connectivity matrix of a timeseries array.
  
  Receives a timeseries numpy array and extracts the subject-level connectivity matrix.
  
  Args:
    timeseries: numpy array of shape (n_subjects, n_regions, n_timepoints)
    kind: string, kind of connectivity measure to extract. Defaults to 'correlation'. See
      nilearn.connectome.ConnectivityMeasure for more details.
  
  Returns:
    connectivity: numpy array of shape (n_subjects, n_regions, n_regions)
  """
  
  cm = ConnectivityMeasure(kind=kind)
  
  # transpose {feature <-> sample} and convert to a list of subjects
  X = [sub_ts.T for sub_ts in timeseries]
  y = cm.fit_transform(X)
  return y

# fit connectivity model
connectivities = fit_connectivity(TIMESERIES.values)

# prepare and store the connectivity data
DATASET['connectivity'] = xr.DataArray(connectivities, dims=['subject', 'region', 'region'])
DATASET.attrs['description'] = 'Connectivity matrices extracted from resting timeseries.'

DATASET.to_netcdf('data/julia2018_resting/connectivities.nc', engine='netcdf4')

In [44]:
DATASET

<xarray.Dataset>
Dimensions:         (subject: 32, region: 39, timestep: 125, mni152_coord: 3)
Coordinates:
  * subject         (subject) object 'AVGP01' 'AVGP02' ... 'NVGP19' 'NVGP19NEW'
  * region          (region) object 'R IPS' 'L IPS' ... 'L midtemporal' 'vmPFC'
  * mni152_coord    (mni152_coord) object 'x' 'y' 'z'
  * timestep        (timestep) int64 0 1 2 3 4 5 6 ... 119 120 121 122 123 124
Data variables:
    timeseries      (subject, region, timestep) float64 -0.04544 ... 0.1655
    subject_groups  (subject) object ...
    regions         (region, mni152_coord) int64 ...
    connectivity    (subject, region, region) float64 1.0 0.5569 ... 0.2011 1.0
Attributes:
    description:  Connectivity matrices extracted from resting timeseries.